## Churn data processing

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# === EDIT THESE ===
PROJECT_DIR   = "path"  # where outputs/scripts will go
DATASET_PATH  = "path"
REPORTS_JSONL = "path"

# Make project directories
import os
os.makedirs(PROJECT_DIR, exist_ok=True)
os.makedirs(f"{PROJECT_DIR}/artifacts", exist_ok=True)
os.chdir(PROJECT_DIR)

print("Working in:", os.getcwd())
print("Dataset:", DATASET_PATH)
print("Reports:", REPORTS_JSONL)


In [ ]:
# Install dependencies
!pip -q install -U pip
!pip -q install pandas numpy scikit-learn joblib lightgbm catboost shap tqdm

In [ ]:
%%writefile train_churn.py
import argparse, json, os, warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss, classification_report
import joblib

try:
    from catboost import CatBoostClassifier
except:
    CatBoostClassifier = None
try:
    from lightgbm import LGBMClassifier
except:
    LGBMClassifier = None

LABEL_COL = "Churn_Status"
DATE_COLS = ["First_Policy_Date_Gre","Last_Renewal_Date_Gre","Last_Expiration_Date"]
NUM_SAFE_NONNEG = ["Years_With_Company","Total_Premium_Paid","Renewal_Count","Avg_Claim_Amount","Claim_Count"]

def _parse_dates(df):
    for c in DATE_COLS:
        df[c] = pd.to_datetime(df[c], errors="coerce", infer_datetime_format=True)
    return df

def _winsorize(s, p=0.995):
    if s.isna().all(): return s
    hi = s.quantile(p); lo = s.quantile(1-p)
    return s.clip(lower=lo, upper=hi)

def _clean_and_engineer(df):
    key_cols = [c for c in ["Customer_ID","Last_Expiration_Date"] if c in df.columns]
    df = df.drop_duplicates(subset=key_cols if key_cols else None, keep="last").copy()
    df = _parse_dates(df)
    for c in NUM_SAFE_NONNEG:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
            df.loc[df[c] < 0, c] = np.nan
    for c in ["Total_Premium_Paid","Avg_Claim_Amount"]:
        if c in df.columns:
            df[c] = _winsorize(df[c].astype(float), p=0.995)

    ref_date = df["Last_Expiration_Date"].max()
    ref_date = pd.Timestamp(ref_date) if pd.notnull(ref_date) else pd.Timestamp.today().normalize()

    feat = pd.DataFrame(index=df.index)
    m = {
        "years_with_company":"Years_With_Company",
        "total_premium_paid":"Total_Premium_Paid",
        "renewal_count":"Renewal_Count",
        "avg_claim_amount":"Avg_Claim_Amount",
        "claim_count":"Claim_Count",
    }
    for new, old in m.items():
        feat[new] = df[old].astype(float) if old in df.columns else np.nan

    feat["days_since_first_policy"] = (ref_date - df["First_Policy_Date_Gre"]).dt.days
    feat["days_since_last_renewal"] = (ref_date - df["Last_Renewal_Date_Gre"]).dt.days
    feat["days_to_expiration"]      = (df["Last_Expiration_Date"] - ref_date).dt.days

    feat = feat.replace([np.inf,-np.inf], np.nan)
    medians = feat.median(numeric_only=True).to_dict()
    feat = feat.fillna(medians)

    feat["days_since_first_policy"] = feat["days_since_first_policy"].clip(-365*5, 365*50)
    feat["days_since_last_renewal"] = feat["days_since_last_renewal"].clip(-365*5, 365*5)
    feat["days_to_expiration"]      = feat["days_to_expiration"].clip(-365*5, 365*5)

    assert LABEL_COL in df.columns, f"Label '{LABEL_COL}' not found"
    y = df[LABEL_COL].astype(int).values
    return feat, y, medians

def _fit_model(algo, X, y):
    if algo=="catboost":
        if CatBoostClassifier is None: raise RuntimeError("catboost not installed")
        m = CatBoostClassifier(loss_function="Logloss", eval_metric="AUC",
                               depth=6, learning_rate=0.08, iterations=800,
                               l2_leaf_reg=3.0, random_seed=42, verbose=False)
        m.fit(X, y)
        return m
    if algo=="lightgbm":
        if LGBMClassifier is None: raise RuntimeError("lightgbm not installed")
        m = LGBMClassifier(objective="binary", boosting_type="gbdt",
                           num_leaves=63, learning_rate=0.05, n_estimators=1200,
                           reg_lambda=1.0, subsample=0.9, colsample_bytree=0.9,
                           random_state=42, n_jobs=-1)
        m.fit(X, y)
        return m
    raise ValueError("algo must be catboost or lightgbm")

def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--csv", required=True)
    ap.add_argument("--algo", choices=["catboost","lightgbm"], required=True)
    ap.add_argument("--test_size", type=float, default=0.2)
    ap.add_argument("--out_dir", default="artifacts")
    args = ap.parse_args()

    os.makedirs(args.out_dir, exist_ok=True)
    df = pd.read_csv(args.csv)
    X, y, med = _clean_and_engineer(df)

    Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=args.test_size, stratify=y, random_state=42)

    base = _fit_model(args.algo, Xtr, ytr)
    cal = CalibratedClassifierCV(base, method="isotonic", cv=3).fit(Xtr, ytr)

    p = cal.predict_proba(Xte)[:,1]
    auc = roc_auc_score(yte, p); ap_ = average_precision_score(yte, p); brier = brier_score_loss(yte, p)
    print(f"[{args.algo}] AUC={auc:.4f} | AP={ap_:.4f} | Brier={brier:.5f}")
    print(classification_report(yte, (p>=0.5).astype(int), digits=3))

    joblib.dump(cal, f"{args.out_dir}/churn_{args.algo}_calibrated.joblib")
    with open(f"{args.out_dir}/churn_preprocess_artifacts.json","w",encoding="utf-8") as f:
        json.dump({"feature_order": list(X.columns), "medians": med}, f, ensure_ascii=False, indent=2)

    if hasattr(base,"feature_importances_"):
        imp = pd.Series(base.feature_importances_, index=X.columns).sort_values(ascending=False)
        imp.to_csv(f"{args.out_dir}/feature_importance_{args.algo}.csv")
        print("Top features:\n", imp.head(10))

    print("DONE")

if __name__=="__main__":
    main()

In [ ]:
# Train both models (choose the better one by AUC/AP/Brier)
!python train_churn.py --csv "{DATASET_PATH}" --algo catboost  --out_dir artifacts
!python train_churn.py --csv "{DATASET_PATH}" --algo lightgbm --out_dir artifacts

In [ ]:
%%writefile score_bulk.py
import json, joblib, argparse
import numpy as np, pandas as pd

def load_artifacts(model_path, art_path):
    model = joblib.load(model_path)
    art = json.load(open(art_path, "r", encoding="utf-8"))
    return model, art["feature_order"], art["medians"]

def engineer(df):
    df = df.copy()
    for c in ["First_Policy_Date_Gre","Last_Renewal_Date_Gre","Last_Expiration_Date"]:
        df[c] = pd.to_datetime(df[c], errors="coerce", infer_datetime_format=True)
    ref = df["Last_Expiration_Date"].max()
    ref = pd.Timestamp(ref) if pd.notnull(ref) else pd.Timestamp.today().normalize()

    X = pd.DataFrame(index=df.index)
    X["years_with_company"] = df["Years_With_Company"]
    X["total_premium_paid"] = df["Total_Premium_Paid"]
    X["renewal_count"]      = df["Renewal_Count"]
    X["avg_claim_amount"]   = df["Avg_Claim_Amount"]
    X["claim_count"]        = df["Claim_Count"]
    X["days_since_first_policy"] = (ref - df["First_Policy_Date_Gre"]).dt.days
    X["days_since_last_renewal"] = (ref - df["Last_Renewal_Date_Gre"]).dt.days
    X["days_to_expiration"]      = (df["Last_Expiration_Date"] - ref).dt.days

    X = X.replace([np.inf,-np.inf], np.nan)
    return X, ref

def unwrap_tree_model(model):
    # Unwrap scikit's CalibratedClassifierCV to the underlying tree model (LightGBM/CatBoost)
    # Supported patterns across sklearn versions:
    # 1) model.base_estimator
    # 2) model.calibrated_classifiers_[0].estimator
    if model.__class__.__name__ == "CalibratedClassifierCV":
        if hasattr(model, "base_estimator") and model.base_estimator is not None:
            return model.base_estimator
        if hasattr(model, "calibrated_classifiers_") and len(model.calibrated_classifiers_) > 0:
            cc = model.calibrated_classifiers_[0]
            if hasattr(cc, "estimator"):
                return cc.estimator
    return model  # already a native tree model

def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--csv_in", required=True)
    ap.add_argument("--csv_out", default="artifacts/churn_scored.csv")
    ap.add_argument("--model_path", required=True)
    ap.add_argument("--art_path", default="artifacts/churn_preprocess_artifacts.json")
    ap.add_argument("--topk", type=int, default=3)
    ap.add_argument("--with_shap", action="store_true")
    ap.add_argument("--shap_sample", type=int, default=0, help="0=all rows, otherwise sample N rows for SHAP")
    args = ap.parse_args()

    model, feats, med = load_artifacts(args.model_path, args.art_path)
    df = pd.read_csv(args.csv_in)
    X, ref = engineer(df)
    X = X[feats].fillna(med)

    probs = model.predict_proba(X)[:,1]
    seg = np.where(probs>=0.7,"High", np.where(probs>=0.4,"Mid","Low"))

    out = df.copy()
    out["p_churn"] = probs
    out["segment"] = seg

    if args.with_shap:
        import shap

        base = unwrap_tree_model(model)
        # Safety: if SHAP still doesn't support the unwrapped model, skip gracefully
        try:
            explainer = shap.TreeExplainer(base)
        except Exception as e:
            print(f"[WARN] SHAP TreeExplainer unsupported for model={type(base)}; skipping SHAP. Error: {e}")
            out["top_reasons"] = ""
            out.to_csv(args.csv_out, index=False)
            print("Saved:", args.csv_out)
            return

        if args.shap_sample and args.shap_sample < len(X):
            samp_idx = np.random.RandomState(42).choice(len(X), size=args.shap_sample, replace=False)
            X_shap = X.iloc[samp_idx]
            index_target = X.index[samp_idx]
        else:
            X_shap = X
            index_target = X.index

        sv = explainer.shap_values(X_shap)
        if isinstance(sv, list):  # e.g., [neg_class, pos_class]
            sv = sv[1]
        vals = np.abs(sv)

        top_names = []
        for i in range(X_shap.shape[0]):
            idx = np.argsort(-vals[i])[:args.topk]
            top_names.append(",".join([X_shap.columns[j] for j in idx]))

        out["top_reasons"] = ""
        out.loc[index_target, "top_reasons"] = top_names
    else:
        out["top_reasons"] = ""

    out.to_csv(args.csv_out, index=False)
    print("Saved:", args.csv_out)

if __name__=="__main__":
    main()

Overwriting score_bulk.py


In [ ]:
# Score full dataset (set model to the winner: lightgbm or catboost)
MODEL_PATH = f"{PROJECT_DIR}/path"  # change if catboost won
ART_PATH   = f"{PROJECT_DIR}/path"

!python score_bulk.py --csv_in "{DATASET_PATH}" --model_path "{MODEL_PATH}" --art_path "{ART_PATH}" --with_shap

In [ ]:
%%writefile build_sft_dataset.py
import json, argparse
import pandas as pd
import numpy as np

SYS_PROMPT = (
"sys prompt"
)

def safe_read_reports(path, strict=False, max_warn=20):
    reports, warns = [], 0
    with open(path, "r", encoding="utf-8-sig") as f:
        data = f.read()

    first_non_ws = next((ch for ch in data if not ch.isspace()), "")
    if first_non_ws == "[":  # JSON array file
        try:
            parsed = json.loads(data)
            if isinstance(parsed, list):
                return parsed
            else:
                raise ValueError("Top-level JSON must be a list.")
        except Exception as e:
            raise ValueError(f"Failed to parse JSON array file: {e}")

    # JSONL fallback
    with open(path, "r", encoding="utf-8-sig") as f:
        for i, line in enumerate(f, start=1):
            s = line.strip()
            if not s or s.startswith("#") or s.startswith("//"):
                continue
            try:
                reports.append(json.loads(s))
            except Exception as e:
                warns += 1
                msg = f"[WARN] bad JSON at line {i}: {e}\nLINE: {s[:200]}"
                if strict:
                    raise ValueError(msg)
                if warns <= max_warn:
                    print(msg)
                continue
    return reports

def get_nested(d, *keys, default=None):
    cur = d
    try:
        for k in keys:
            if cur is None: return default
            cur = cur.get(k)
        return cur if cur is not None else default
    except AttributeError:
        return default

def to_example(report, churn_row):
    inp = {
      "customer_profile": {
        "years_with_company": get_nested(report,"input","years_with_company", default=get_nested(report,"input","Years_With_Company")),
        "total_premium_paid": get_nested(report,"input","total_premium_paid"),
        "renewal_count":      get_nested(report,"input","renewal_count"),
        "avg_claim_amount":   get_nested(report,"input","avg_claim_amount"),
        "claim_count":        get_nested(report,"input","claim_count"),
        "days_since_first_policy": get_nested(report,"input","days_since_first_policy"),
        "days_since_last_renewal": get_nested(report,"input","days_since_last_renewal"),
        "days_to_expiration":      get_nested(report,"input","days_to_expiration")
      },
      "underwriting": {
        "risk_score": report.get("risk_score"),
        "risk_level": report.get("risk_level"),
        "coverage": get_nested(report,"input","coverage")
      },
      "churn": {
        "p_churn": float(churn_row.get("p_churn", 0.0)) if isinstance(churn_row.get("p_churn", 0.0),(int,float,str)) else 0.0,
        "segment": churn_row.get("segment",""),
        "top_reasons": (str(churn_row.get("top_reasons","")).split(",") if pd.notna(churn_row.get("top_reasons","")) else [])
      },
      "constraints": {
        "max_discount_pct": number,
        "allowed_actions": ["actions"]
      }
    }
    rec_conditions = report.get("recommended_conditions", [])
    if not isinstance(rec_conditions, list):
        rec_conditions = []
    retention_plan = [{"step": i+1, "action": a} for i,a in enumerate(rec_conditions)]
    if not retention_plan:
        rpa = report.get("recommended_premium_action", "")
        if rpa:
            retention_plan = [{"step":1,"action": rpa}]

    out = {
      "summary": report.get("short_summary",""),
      "retention_plan": retention_plan,
      "underwriting_notes": [report.get("decision_rationale","")] if report.get("decision_rationale") else [],
      "cx_message_short": report.get("short_summary","")
    }

    return {
      "messages":[
        {"role":"system","content":SYS_PROMPT},
        {"role":"user","content":json.dumps(inp, ensure_ascii=False)},
        {"role":"assistant","content":json.dumps(out, ensure_ascii=False)}
      ]
    }

def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--reports_jsonl", required=True)
    ap.add_argument("--churn_scored_csv", required=True)
    ap.add_argument("--out_jsonl", default="artifacts/llm_sft_dataset.jsonl")
    ap.add_argument("--join_key", default=None)
    ap.add_argument("--strict_reports", action="store_true")
    args = ap.parse_args()

    churn = pd.read_csv(args.churn_scored_csv).reset_index(drop=True)
    reports = safe_read_reports(args.reports_jsonl, strict=args.strict_reports)

    data = []
    if args.join_key and args.join_key in churn.columns:
        idx_map = {row[args.join_key]: i for i,row in churn.iterrows()}
        matched = 0
        for rep in reports:
            key = get_nested(rep,"input",args.join_key, default=rep.get(args.join_key))
            if key in idx_map:
                data.append(to_example(rep, churn.loc[idx_map[key]]))
                matched += 1
        print(f"[INFO] Keyed join matched {matched}/{len(reports)}")
    else:
        n = min(len(reports), len(churn))
        for i in range(n):
            data.append(to_example(reports[i], churn.iloc[i]))
        if len(reports) != len(churn):
            print(f"[INFO] Index-join used: reports={len(reports)} churn={len(churn)} -> n={n}")

    with open(args.out_jsonl,"w",encoding="utf-8") as f:
        for r in data:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")
    print("Saved:", args.out_jsonl, "count:", len(data))

if __name__=="__main__":
    main()

In [ ]:
# Build SFT dataset (pre fine-tuning artifact)
!python build_sft_dataset.py --reports_jsonl "{REPORTS_JSONL}" --churn_scored_csv "{PROJECT_DIR}/artifacts/churn_scored.csv" --out_jsonl "{PROJECT_DIR}/artifacts/llm_sft_dataset.jsonl"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from pathlib import Path
import os

PROJECT_DIR   = Path("path")
DATASET_PATH  = Path("path")
REPORTS_JSONL = Path("path")

# create folders
PROJECT_DIR.mkdir(parents=True, exist_ok=True)
(PROJECT_DIR / "artifacts").mkdir(exist_ok=True)

print("PROJECT_DIR exists:", PROJECT_DIR.exists())
print("Dataset exists:", DATASET_PATH.exists())
print("Reports exists:", REPORTS_JSONL.exists())

# cd using pure Python (avoids comment/space issues)
os.chdir(str(PROJECT_DIR))
print("Working in:", os.getcwd())

# quick sanity checks (raise if paths are wrong)
assert (PROJECT_DIR / "score_bulk.py").exists(), "score_bulk.py not found in PROJECT_DIR"
assert DATASET_PATH.exists(), "Dataset path not found"
assert REPORTS_JSONL.exists(), "reports.json not found"


In [11]:
# Install required libs for loading & SHAP (optional)
!pip -q install catboost shap pandas numpy scikit-learn joblib

# Quick sanity check
import catboost, sklearn, shap, pandas as pd, numpy as np, joblib, sys
print("catboost:", catboost.__version__)
print("python:", sys.version)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 7.2 MB/s eta 0:00:00
catboost: 1.2.8
python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]


In [ ]:
# Uses the calibrated CatBoost you trained: churn_catboost_calibrated.joblib
!python "score_bulk.py" \
  --csv_in "{DATASET_PATH}" \
  --model_path "{PROJECT_DIR}/artifacts/churn_catboost_calibrated.joblib" \
  --art_path   "{PROJECT_DIR}/artifacts/churn_preprocess_artifacts.json" \
  --with_shap --shap_sample 5000

In [ ]:
%%writefile "policy_engine.py"
def retention_policy(p_churn, risk_score, max_discount=10):
    seg = "High" if p_churn>=0.7 else "Mid" if p_churn>=0.4 else "Low"
    ladder = {
        "Low": ["values"],
        "Mid": ["values"],
        "High":["values"]
    }[seg]
    # risk gate: reduce discount cap for high-risk underwriting
    cap = max_discount if risk_score<=60 else max(0, max_discount-4)
    return {"segment": seg, "allowed_actions": ladder, "max_discount_pct": cap}

In [ ]:
!python "build_sft_dataset.py" \
  --reports_jsonl "{REPORTS_JSONL}" \
  --churn_scored_csv "{PROJECT_DIR}/artifacts/churn_scored.csv" \
  --out_jsonl "{PROJECT_DIR}/artifacts/llm_sft_dataset.jsonl"